# 1. Install and Import Dependencies

In [25]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.8.1+cu111 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.

In [26]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [28]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [29]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [30]:
result = model(tokens)

In [31]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [32]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [33]:
#r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
#soup = BeautifulSoup(r.text, 'html.parser')
#regex = re.compile('.*comment.*')
#results = soup.find_all('p', {'class':regex})
#reviews = [result.text for result in results]

In [44]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

# Fetch the data from the URL
source = requests.get('https://kith.com/products.json').text

# Parse the HTML content
soup = BeautifulSoup(source, 'lxml')

# Find the 'p' tag and load its JSON content
p = soup.find('p')
product_data = json.loads(p.text)

# Create an empty DataFrame with appropriate columns
df = pd.DataFrame(columns=['ID', 'Reviews', 'Handle'])

# Iterate through all products and add rows to the DataFrame
for product in product_data["products"]:
    id = product["id"]
    reviews = product["title"]
    handle = product["handle"]

    # Append the product details as a new row in the DataFrame
    df = pd.concat([df, pd.DataFrame({'ID': [id], 'Reviews': [reviews], 'Handle': [handle]})], ignore_index=True)

# Print the DataFrame
print(df)


               ID                                            Reviews  \
0   8284961800320  The 8th St Gazelle Indoor by Ronnie Fieg for a...   
1   8284961767552  The 8th St BW Army by Ronnie Fieg for adidas O...   
2   8284961833088  The 8th St AS350 by Ronnie Fieg for adidas Ori...   
3   8284946727040              Birkenstock Kids Tokio Bikibuc - Navy   
4   8284946694272             Birkenstock Kids Tokio Bikibuc - Mocha   
5   8284946563200  adidas Samba Crib - Cloud White / Core Black /...   
6   8284946530432  adidas WMNS Sambae - Cloud White / Core Black ...   
7   8284946399360            adidas Country Japan - Core Black / Gum   
8   8284803399808  Salomon x MM6 Maison Margiela WMNS ACS Pro - S...   
9   8284786557056  adidas Handball Spezial - Core Black / Cloud W...   
10  8284627730560  adidas Kids Samba OG - White / Semi Pink Spark...   
11  8284598009984  adidas Adizero PR - Crystal White / Cloud Whit...   
12  8284597977216   adidas Adizero Evo SL - Core Black / Cloud W

# 5. Load Reviews into DataFrame and Score

In [45]:
import numpy as np
import pandas as pd

In [46]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [48]:
df['sentiment'] = df['Reviews'].apply(lambda x: sentiment_score(x))

In [49]:
df.head()

,ID,Reviews,Handle,sentiment
0,8284961800320,The 8th St Gazelle Indoor by Ronnie Fieg for a...,aajs1754-ph,5
1,8284961767552,The 8th St BW Army by Ronnie Fieg for adidas O...,aajr5306-ph,5
2,8284961833088,The 8th St AS350 by Ronnie Fieg for adidas Ori...,aajs1712-ph,5
3,8284946727040,Birkenstock Kids Tokio Bikibuc - Navy,br1027917,3
4,8284946694272,Birkenstock Kids Tokio Bikibuc - Mocha,br1027916,4
